In [ ]:
!pip install ultralytics # Ensure ultralytics is installed

In [ ]:
import cv2
from ultralytics import YOLO

# Instantiate the YOLO model
model = YOLO("yolov8s.pt")

# Get the index for the 'person' class from the model.names dictionary
try:
    person_class_id = [key for key, val in model.names.items() if val == 'person'][0]
except IndexError:
    print(f"Error: 'person' class not found in model names.")
    exit()

video_path = input("Please enter the video filename (e.g., video1.mp4), or press Enter for default: ") or "video1.mp4"
cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print(f"Error: Could not open video file: {video_path}")
    exit()

# Initialize an empty set to store unique track IDs of detected people
unique_person_ids = set()

# Get video properties for output video writer
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Initialize output video writer
output_video_path = 'output_video_with_detections.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Codec for .mp4 file
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

if not out.isOpened():
    print(f"Error: Could not open video writer for {output_video_path}")
    exit()

# Initialize frame counters
processed_frames_count = 0
dropped_frames_count = 0
is_end_of_stream = False # New flag to differentiate end of stream

# Start a while loop that continues as long as cap.isOpened() is True
while cap.isOpened():
    # Read a frame from the video capture object
    ret, frame = cap.read()

    # Check if the frame was read successfully
    if not ret:
        # If not successful, check if it's the actual end of the stream or a dropped frame
        if cap.get(cv2.CAP_PROP_POS_FRAMES) == cap.get(cv2.CAP_PROP_FRAME_COUNT):
            is_end_of_stream = True
            print("Reached end of video stream.")
        else:
            dropped_frames_count += 1
            print(f"Error: Dropped frame detected. Dropped frames total: {dropped_frames_count}")
        break

    processed_frames_count += 1

    # Perform object detection and tracking on the current frame
    # Only track 'person' class with persist=True for continuous tracking
    results = model.track(frame, persist=True, classes=person_class_id, verbose=False)[0]

    # Initialize current frame's detected track IDs
    current_frame_person_ids = set()

    # Iterate through the boxes attribute of the results object
    if results.boxes.id is not None:
        for i, box in enumerate(results.boxes):
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cls_id = int(box.cls[0])
            conf = box.conf[0]
            track_id = int(results.boxes.id[i].item())

            # Ensure it's a 'person' detection
            if cls_id == person_class_id:
                current_frame_person_ids.add(track_id)
                unique_person_ids.add(track_id)

                # Draw bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2) # Green color

                # Add track ID and confidence score as text
                label = f"ID: {track_id} Conf: {conf:.2f}"
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Add total unique persons count to the frame
    total_unique_text = f"Total Unique Persons: {len(unique_person_ids)}"
    cv2.putText(frame, total_unique_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Write the annotated frame to the output video file
    out.write(frame)

# Release the video capture object and the video writer object
cap.release()
out.release()
print(f"Total frames processed: {processed_frames_count}")
print(f"Total frames dropped during reading: {dropped_frames_count}")
print(f"Total unique persons detected throughout the video: {len(unique_person_ids)}")
print(f"Annotated video saved to: {output_video_path}")

Kérlek add meg a videófájl nevét (pl.: video1.mp4), vagy nyomj Entert az alapértelmezett értékért: 
Elérte a videófolyam végét.
Feldolgozott képkockák száma: 352
Kihagyott képkockák száma az olvasás során: 0
Összes egyedi személy észlelhető a videóban: 35
Annotált videó mentve ide: output_video_with_detections_v2.mp4


# Videó elemzési kód dokumentációja

Ez a kód egy YOLOv8 (You Only Look Once) modellt használ arra, hogy személyeket detektáljon és kövessen egy videóban, majd egy annotált kimeneti videót generáljon, amely jelöli a detektált személyeket és megszámolja az egyedi személyeket a teljes videó során.

## Előfeltételek
*   **ultralytics** könyvtár a YOLOv8 modellhez.
*   **opencv-python** könyvtár a videó beolvasásához és írásához.
*   **yolov8s.pt** modellfájl, amelyet a program automatikusan letölt, ha nincs jelen.

## Használat
1.  **Telepítse a szükséges könyvtárakat:** A kód elején található `!pip install ultralytics` parancs biztosítja az `ultralytics` telepítését.
2.  **Futtassa a kódot:** A kód futtatásakor megkéri a felhasználót, hogy adjon meg egy bemeneti videófájl nevet. Például: `video1.mp4`.
    *   Ha üresen hagyja és Entert nyom, az alapértelmezett `video1.mp4` fájlt használja.
3.  **Bemeneti videó:** Győződjön meg róla, hogy a megadott videófájl (pl. `video1.mp4`) jelen van a Colab környezetben.

## Működés
*   **YOLO modell betöltése:** Betölti a `yolov8s.pt` YOLO modellt.
*   **'person' osztály azonosítása:** Meghatározza a 'person' (személy) osztály indexét a modell által detektálható objektumok közül.
*   **Videó beolvasása:** Megnyitja a felhasználó által megadott videót.
*   **Kimeneti videó írása:** Inicializál egy videóírót `output_video_with_detections_v2.mp4` néven, ahova az annotált képkockákat menti.
*   **Képkocka feldolgozás:** Végighalad a videó összes képkockáján:
    *   Minden képkockán személyeket detektál és követ `persist=True` beállítással, hogy megőrizze a követési azonosítókat a képkockák között.
    *   Rajzol egy zöld határolókeretet a detektált személyek köré, és hozzáadja a követési azonosítójukat (`ID`) és megbízhatósági pontszámukat (`Conf`).
    *   Nyomon követi az **egyedi személyek számát** a videóban (minden detektált személyt csak egyszer számol).
    *   Megjeleníti a **feldolgozott képkockák számát** és a **kihagyott képkockák számát**.
    *   Különbséget tesz a valódi stream vége és a kihagyott képkockák között a `cv2.CAP_PROP_POS_FRAMES` és `cv2.CAP_PROP_FRAME_COUNT` segítségével.
*   **Eredmények kiírása:** A feldolgozás befejeztével a program kiírja a feldolgozott képkockák számát, a kihagyott képkockák számát, az összes egyedi detektált személy számát és a kimeneti videó elérési útját.